In [77]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [78]:
ais_test = pd.read_csv('ais_test.csv')
ais_test

#Hei på deg, dette er en test

,ID,vesselId,time,scaling_factor
0,0,61e9f3aeb937134a3c4bfe3d,2024-05-08 00:03:16,0.3
1,1,61e9f473b937134a3c4c02df,2024-05-08 00:06:17,0.3
2,2,61e9f469b937134a3c4c029b,2024-05-08 00:10:02,0.3
3,3,61e9f45bb937134a3c4c0221,2024-05-08 00:10:34,0.3
4,4,61e9f38eb937134a3c4bfd8d,2024-05-08 00:12:27,0.3
...,...,...,...,...
51734,51734,61e9f3a8b937134a3c4bfdf3,2024-05-12 23:59:58,0.1
51735,51735,61e9f3b4b937134a3c4bfe77,2024-05-12 23:59:58,0.1
51736,51736,61e9f46cb937134a3c4c02b7,2024-05-12 23:59:58,0.1
51737,51737,61e9f465b937134a3c4c0269,2024-05-12 23:59:58,0.1


In [79]:
ais_train = pd.read_csv('first_50000_rows.csv', sep='|')
ais_train

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3
...,...,...,...,...,...,...,...,...,...,...,...
49995,2024-01-05 07:36:12,5.5,6.1,0,0,0,01-04 20:00,-24.03371,-46.34749,61e9f410b937134a3c4c0049,61d36fdf0a1807568ff9a1b0
49996,2024-01-05 07:36:13,94.9,0.0,0,31,5,01-02 18:00,39.64086,-0.22345,61e9f468b937134a3c4c0289,61d37fb629b60f6113c89e99
49997,2024-01-05 07:36:16,221.9,17.8,0,223,0,01-11 22:00,34.46860,138.21546,61e9f3aab937134a3c4bfe0f,61d37a221366c3998241d928
49998,2024-01-05 07:36:16,324.1,0.0,0,268,5,01-03 20:30,53.33446,7.16211,61e9f397b937134a3c4bfdaf,61d375e893c6feb83e5eb3e4


In [80]:
# Merge test set with training set history
ais_train['time'] = pd.to_datetime(ais_train['time'])  # Ensure time is datetime
ais_test['time'] = pd.to_datetime(ais_test['time'])

# Check for missing values
print(ais_train.isnull().sum())
print(ais_train.info())

time           0
cog            0
sog            0
rot            0
heading        0
navstat        0
etaRaw         0
latitude       0
longitude      0
vesselId       0
portId       203
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       50000 non-null  datetime64[ns]
 1   cog        50000 non-null  float64       
 2   sog        50000 non-null  float64       
 3   rot        50000 non-null  int64         
 4   heading    50000 non-null  int64         
 5   navstat    50000 non-null  int64         
 6   etaRaw     50000 non-null  object        
 7   latitude   50000 non-null  float64       
 8   longitude  50000 non-null  float64       
 9   vesselId   50000 non-null  object        
 10  portId     49797 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(3)
memory usage: 4.2+ MB

In [81]:
# Sort by vesselId and time
ais_train = ais_train.sort_values(['vesselId', 'time'])
ais_train

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
65,2024-01-01 00:14:36,348.0,0.0,0,333,5,12-29 21:00,51.30883,3.23027,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156
218,2024-01-01 00:35:36,8.0,0.0,0,333,5,12-29 21:00,51.30882,3.23025,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156
378,2024-01-01 00:56:34,20.0,0.0,0,333,5,12-29 21:00,51.30882,3.23027,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156
526,2024-01-01 01:17:35,6.0,0.0,0,334,5,12-29 21:00,51.30880,3.23023,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156
612,2024-01-01 01:35:36,353.0,0.0,0,334,5,12-29 21:00,51.30882,3.23030,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156
...,...,...,...,...,...,...,...,...,...,...,...
49242,2024-01-05 05:56:19,0.0,0.0,-2,135,5,01-05 05:00,53.86176,8.72497,clh6aqawa0007gh0z9h6zi9bo,61d375e793c6feb83e5eb3e3
49468,2024-01-05 06:17:19,0.0,0.0,3,135,5,01-05 05:00,53.86175,8.72496,clh6aqawa0007gh0z9h6zi9bo,61d375e793c6feb83e5eb3e3
49510,2024-01-05 06:35:19,0.0,0.0,-2,135,5,01-05 05:00,53.86178,8.72498,clh6aqawa0007gh0z9h6zi9bo,61d375e793c6feb83e5eb3e3
49694,2024-01-05 06:56:19,0.0,0.0,2,134,5,01-05 05:00,53.86178,8.72498,clh6aqawa0007gh0z9h6zi9bo,61d375e793c6feb83e5eb3e3


In [82]:
ais_test = ais_test.sort_values(['vesselId', 'time'])

# label where data comes from before concatenation
ais_train['dataset'] = 'train'
ais_test['dataset'] = 'test'
ais_test

,ID,vesselId,time,scaling_factor,dataset
4,4,61e9f38eb937134a3c4bfd8d,2024-05-08 00:12:27,0.3,test
201,201,61e9f38eb937134a3c4bfd8d,2024-05-08 00:39:27,0.3,test
583,583,61e9f38eb937134a3c4bfd8d,2024-05-08 01:33:28,0.3,test
701,701,61e9f38eb937134a3c4bfd8d,2024-05-08 01:51:26,0.3,test
829,829,61e9f38eb937134a3c4bfd8d,2024-05-08 02:03:29,0.3,test
...,...,...,...,...,...
51161,51161,clh6aqawa0007gh0z9h6zi9bo,2024-05-12 22:37:33,0.1,test
51302,51302,clh6aqawa0007gh0z9h6zi9bo,2024-05-12 22:58:05,0.1,test
51444,51444,clh6aqawa0007gh0z9h6zi9bo,2024-05-12 23:18:20,0.1,test
51595,51595,clh6aqawa0007gh0z9h6zi9bo,2024-05-12 23:39:21,0.1,test


In [83]:
# Combine training and test data into a single DataFrame
combined_df = pd.concat([ais_train, ais_test], ignore_index=True)

# Sort by vesselId and time to ensure proper order for calculating lags
combined_df = combined_df.sort_values(['vesselId', 'time'])
combined_df

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,dataset,ID,scaling_factor
0,2024-01-01 00:14:36,348.0,0.0,0.0,333.0,5.0,12-29 21:00,51.30883,3.23027,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN
1,2024-01-01 00:35:36,8.0,0.0,0.0,333.0,5.0,12-29 21:00,51.30882,3.23025,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN
2,2024-01-01 00:56:34,20.0,0.0,0.0,333.0,5.0,12-29 21:00,51.30882,3.23027,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN
3,2024-01-01 01:17:35,6.0,0.0,0.0,334.0,5.0,12-29 21:00,51.30880,3.23023,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN
4,2024-01-01 01:35:36,353.0,0.0,0.0,334.0,5.0,12-29 21:00,51.30882,3.23030,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101734,2024-05-12 22:37:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51161.0,0.1
101735,2024-05-12 22:58:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51302.0,0.1
101736,2024-05-12 23:18:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51444.0,0.1
101737,2024-05-12 23:39:21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51595.0,0.1


In [84]:
# Create lag features (e.g., lag of 1, 2, 3 steps)
for lag in [1, 2, 3]:
    combined_df[f'sog_lag{lag}'] = combined_df.groupby('vesselId')['sog'].shift(lag).ffill()
    combined_df[f'cog_lag{lag}'] = combined_df.groupby('vesselId')['cog'].shift(lag).ffill()
    combined_df[f'rot_lag{lag}'] = combined_df.groupby('vesselId')['rot'].shift(lag).ffill()
    combined_df[f'heading_lag{lag}'] = combined_df.groupby('vesselId')['heading'].shift(lag).ffill()
    combined_df[f'navstat_lag{lag}'] = combined_df.groupby('vesselId')['navstat'].shift(lag).ffill()
    combined_df[f'etaRaw_lag{lag}'] = combined_df.groupby('vesselId')['etaRaw'].shift(lag).ffill()
    combined_df[f'latitude_lag{lag}'] = combined_df.groupby('vesselId')['latitude'].shift(lag).ffill()
    combined_df[f'longitude_lag{lag}'] = combined_df.groupby('vesselId')['longitude'].shift(lag).ffill()
    
# Adding moving averages on a set of attributes
combined_df['sog_ma5'] = combined_df.groupby('vesselId')['sog'].rolling(window=5, min_periods=1).apply(lambda x: x.dropna().mean()).reset_index(0, drop=True)
combined_df['cog_ma5'] = combined_df.groupby('vesselId')['cog'].rolling(window=5, min_periods=1).apply(lambda x: x.dropna().mean()).reset_index(0, drop=True)
combined_df['rot_ma5'] = combined_df.groupby('vesselId')['rot'].rolling(window=5, min_periods=1).apply(lambda x: x.dropna().mean()).reset_index(0, drop=True)
combined_df['heading_ma5'] = combined_df.groupby('vesselId')['heading'].rolling(window=5, min_periods=1).apply(lambda x: x.dropna().mean()).reset_index(0, drop=True)
combined_df['latitude_ma5'] = combined_df.groupby('vesselId')['latitude'].rolling(window=5, min_periods=1).apply(lambda x: x.dropna().mean()).reset_index(0, drop=True)
combined_df['longitude_ma5'] = combined_df.groupby('vesselId')['longitude'].rolling(window=5, min_periods=1).apply(lambda x: x.dropna().mean()).reset_index(0, drop=True)

combined_df

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,dataset,ID,scaling_factor,sog_lag1,cog_lag1,rot_lag1,heading_lag1,navstat_lag1,etaRaw_lag1,latitude_lag1,longitude_lag1,sog_lag2,cog_lag2,rot_lag2,heading_lag2,navstat_lag2,etaRaw_lag2,latitude_lag2,longitude_lag2,sog_lag3,cog_lag3,rot_lag3,heading_lag3,navstat_lag3,etaRaw_lag3,latitude_lag3,longitude_lag3,sog_ma5,cog_ma5,rot_ma5,heading_ma5,latitude_ma5,longitude_ma5
0,2024-01-01 00:14:36,348.0,0.0,0.0,333.0,5.0,12-29 21:00,51.30883,3.23027,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,348.000000,0.0,333.00,51.308830,3.230270
1,2024-01-01 00:35:36,8.0,0.0,0.0,333.0,5.0,12-29 21:00,51.30882,3.23025,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN,0.0,348.0,0.0,333.0,5.0,12-29 21:00,51.30883,3.23027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,178.000000,0.0,333.00,51.308825,3.230260
2,2024-01-01 00:56:34,20.0,0.0,0.0,333.0,5.0,12-29 21:00,51.30882,3.23027,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN,0.0,8.0,0.0,333.0,5.0,12-29 21:00,51.30882,3.23025,0.0,348.0,0.0,333.0,5.0,12-29 21:00,51.30883,3.23027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,125.333333,0.0,333.00,51.308823,3.230263
3,2024-01-01 01:17:35,6.0,0.0,0.0,334.0,5.0,12-29 21:00,51.30880,3.23023,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN,0.0,20.0,0.0,333.0,5.0,12-29 21:00,51.30882,3.23027,0.0,8.0,0.0,333.0,5.0,12-29 21:00,51.30882,3.23025,0.0,348.0,0.0,333.0,5.0,12-29 21:00,51.30883,3.23027,0.0,95.500000,0.0,333.25,51.308817,3.230255
4,2024-01-01 01:35:36,353.0,0.0,0.0,334.0,5.0,12-29 21:00,51.30882,3.23030,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN,0.0,6.0,0.0,334.0,5.0,12-29 21:00,51.30880,3.23023,0.0,20.0,0.0,333.0,5.0,12-29 21:00,51.30882,3.23027,0.0,8.0,0.0,333.0,5.0,12-29 21:00,51.30882,3.23025,0.0,147.000000,0.0,333.40,51.308818,3.230264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101734,2024-05-12 22:37:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51161.0,0.1,0.0,0.0,1.0,134.0,5.0,01-05 05:00,53.86180,8.72498,0.0,0.0,1.0,134.0,5.0,01-05 05:00,53.86180,8.72498,0.0,0.0,1.0,134.0,5.0,01-05 05:00,53.86180,8.72498,NaN,NaN,NaN,NaN,NaN,NaN
101735,2024-05-12 22:58:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51302.0,0.1,0.0,0.0,1.0,134.0,5.0,01-05 05:00,53.86180,8.72498,0.0,0.0,1.0,134.0,5.0,01-05 05:00,53.86180,8.72498,0.0,0.0,1.0,134.0,5.0,01-05 05:00,53.86180,8.72498,NaN,NaN,NaN,NaN,NaN,NaN
101736,2024-05-12 23:18:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51444.0,0.1,0.0,0.0,1.0,134.0,5.0,01-05 05:00,53.86180,8.72498,0.0,0.0,1.0,134.0,5.0,01-05 05:00,53.86180,8.72498,0.0,0.0,1.0,134.0,5.0,01-05 05:00,53.86180,8.72498,NaN,NaN,NaN,NaN,NaN,NaN
101737,2024-05-12 23:39:21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51595.0,0.1,0.0,0.0,1.0,134.0,5.0,01-05 05:00,53.86180,8.72498,0.0,0.0,1.0,134.0,5.0,01-05 05:00,53.86180,8.72498,0.0,0.0,1.0,134.0,5.0,01-05 05:00,53.86180,8.72498,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:

# one-hot encode the following navstat lag features
lag_columns = ['navstat_lag1', 'navstat_lag2', 'navstat_lag3']

# One-hot encode the specified columns
combined_df = pd.get_dummies(combined_df, columns=lag_columns, prefix=lag_columns)

# Optionally, inspect the new dataframe
combined_df.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,dataset,ID,scaling_factor,sog_lag1,cog_lag1,rot_lag1,heading_lag1,etaRaw_lag1,latitude_lag1,longitude_lag1,sog_lag2,cog_lag2,rot_lag2,heading_lag2,etaRaw_lag2,latitude_lag2,longitude_lag2,sog_lag3,cog_lag3,rot_lag3,heading_lag3,etaRaw_lag3,latitude_lag3,longitude_lag3,sog_ma5,cog_ma5,rot_ma5,heading_ma5,latitude_ma5,longitude_ma5,navstat_lag1_0.0,navstat_lag1_1.0,navstat_lag1_2.0,navstat_lag1_3.0,navstat_lag1_5.0,navstat_lag1_8.0,navstat_lag1_15.0,navstat_lag2_0.0,navstat_lag2_1.0,navstat_lag2_2.0,navstat_lag2_3.0,navstat_lag2_5.0,navstat_lag2_8.0,navstat_lag2_15.0,navstat_lag3_0.0,navstat_lag3_1.0,navstat_lag3_2.0,navstat_lag3_3.0,navstat_lag3_5.0,navstat_lag3_8.0,navstat_lag3_15.0
0,2024-01-01 00:14:36,348.0,0.0,0.0,333.0,5.0,12-29 21:00,51.30883,3.23027,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,348.000000,0.0,333.00,51.308830,3.230270,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2024-01-01 00:35:36,8.0,0.0,0.0,333.0,5.0,12-29 21:00,51.30882,3.23025,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN,0.0,348.0,0.0,333.0,12-29 21:00,51.30883,3.23027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,178.000000,0.0,333.00,51.308825,3.230260,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2024-01-01 00:56:34,20.0,0.0,0.0,333.0,5.0,12-29 21:00,51.30882,3.23027,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN,0.0,8.0,0.0,333.0,12-29 21:00,51.30882,3.23025,0.0,348.0,0.0,333.0,12-29 21:00,51.30883,3.23027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,125.333333,0.0,333.00,51.308823,3.230263,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3,2024-01-01 01:17:35,6.0,0.0,0.0,334.0,5.0,12-29 21:00,51.30880,3.23023,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN,0.0,20.0,0.0,333.0,12-29 21:00,51.30882,3.23027,0.0,8.0,0.0,333.0,12-29 21:00,51.30882,3.23025,0.0,348.0,0.0,333.0,12-29 21:00,51.30883,3.23027,0.0,95.500000,0.0,333.25,51.308817,3.230255,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False
4,2024-01-01 01:35:36,353.0,0.0,0.0,334.0,5.0,12-29 21:00,51.30882,3.23030,61e9f38eb937134a3c4bfd8d,61d36f9a0a1807568ff9a156,train,NaN,NaN,0.0,6.0,0.0,334.0,12-29 21:00,51.30880,3.23023,0.0,20.0,0.0,333.0,12-29 21:00,51.30882,3.23027,0.0,8.0,0.0,333.0,12-29 21:00,51.30882,3.23025,0.0,147.000000,0.0,333.40,51.308818,3.230264,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False


In [86]:
## Split back


combined_df = combined_df.drop(columns=['ID', 'cog', 'sog', 'rot', 'heading', 'navstat', 'etaRaw', 'latitude', 'longitude', 'portId'])
combined_df

# Separate the enriched test set back from combined data
ais_test_enriched = combined_df[combined_df['dataset'] == 'test'].copy()
ais_train_enriched = combined_df[combined_df['dataset'] == 'train'].copy()

# Drop the 'dataset' column as it's no longer needed
ais_test_enriched = ais_test_enriched.drop(columns=['dataset', ])
ais_train_enriched = ais_train_enriched.drop(columns=['dataset', ])

ais_test_enriched

,time,vesselId,scaling_factor,sog_lag1,cog_lag1,rot_lag1,heading_lag1,etaRaw_lag1,latitude_lag1,longitude_lag1,sog_lag2,cog_lag2,rot_lag2,heading_lag2,etaRaw_lag2,latitude_lag2,longitude_lag2,sog_lag3,cog_lag3,rot_lag3,heading_lag3,etaRaw_lag3,latitude_lag3,longitude_lag3,sog_ma5,cog_ma5,rot_ma5,heading_ma5,latitude_ma5,longitude_ma5,navstat_lag1_0.0,navstat_lag1_1.0,navstat_lag1_2.0,navstat_lag1_3.0,navstat_lag1_5.0,navstat_lag1_8.0,navstat_lag1_15.0,navstat_lag2_0.0,navstat_lag2_1.0,navstat_lag2_2.0,navstat_lag2_3.0,navstat_lag2_5.0,navstat_lag2_8.0,navstat_lag2_15.0,navstat_lag3_0.0,navstat_lag3_1.0,navstat_lag3_2.0,navstat_lag3_3.0,navstat_lag3_5.0,navstat_lag3_8.0,navstat_lag3_15.0
50000,2024-05-08 00:12:27,61e9f38eb937134a3c4bfd8d,0.3,13.2,244.0,0.0,246.0,01-06 11:30,49.4995,-4.18170,13.4,242.0,0.0,246.0,01-06 11:30,49.50783,-4.15792,12.9,243.0,0.0,246.0,01-06 11:30,49.54392,-4.05385,13.200000,241.75,0.0,245.25,49.532125,-4.089072,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False
50001,2024-05-08 00:39:27,61e9f38eb937134a3c4bfd8d,0.3,13.2,244.0,0.0,246.0,01-06 11:30,49.4995,-4.18170,13.2,244.0,0.0,246.0,01-06 11:30,49.49950,-4.18170,13.4,242.0,0.0,246.0,01-06 11:30,49.50783,-4.15792,13.166667,243.00,0.0,246.00,49.517083,-4.131157,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False
50002,2024-05-08 01:33:28,61e9f38eb937134a3c4bfd8d,0.3,13.2,244.0,0.0,246.0,01-06 11:30,49.4995,-4.18170,13.2,244.0,0.0,246.0,01-06 11:30,49.49950,-4.18170,13.2,244.0,0.0,246.0,01-06 11:30,49.49950,-4.18170,13.300000,243.00,0.0,246.00,49.503665,-4.169810,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False
50003,2024-05-08 01:51:26,61e9f38eb937134a3c4bfd8d,0.3,13.2,244.0,0.0,246.0,01-06 11:30,49.4995,-4.18170,13.2,244.0,0.0,246.0,01-06 11:30,49.49950,-4.18170,13.2,244.0,0.0,246.0,01-06 11:30,49.49950,-4.18170,13.200000,244.00,0.0,246.00,49.499500,-4.181700,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False
50004,2024-05-08 02:03:29,61e9f38eb937134a3c4bfd8d,0.3,13.2,244.0,0.0,246.0,01-06 11:30,49.4995,-4.18170,13.2,244.0,0.0,246.0,01-06 11:30,49.49950,-4.18170,13.2,244.0,0.0,246.0,01-06 11:30,49.49950,-4.18170,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101734,2024-05-12 22:37:33,clh6aqawa0007gh0z9h6zi9bo,0.1,0.0,0.0,1.0,134.0,01-05 05:00,53.8618,8.72498,0.0,0.0,1.0,134.0,01-05 05:00,53.86180,8.72498,0.0,0.0,1.0,134.0,01-05 05:00,53.86180,8.72498,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False
101735,2024-05-12 22:58:05,clh6aqawa0007gh0z9h6zi9bo,0.1,0.0,0.0,1.0,134.0,01-05 05:00,53.8618,8.72498,0.0,0.0,1.0,134.0,01-05 05:00,53.86180,8.72498,0.0,0.0,1.0,134.0,01-05 05:00,53.86180,8.72498,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False
101736,2024-05-12 23:18:20,clh6aqawa0007gh0z9h6zi9bo,0.1,0.0,0.0,1.0,134.0,01-05 05:00,53.8618,8.72498,0.0,0.0,1.0,134.0,01-05 05:00,53.86180,8.72498,0.0,0.0,1.0,134.0,01-05 05:00,53.86180,8.72498,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False
101737,2024-05-12 23:39:21,clh6aqawa0007gh0z9h6zi9bo,0.1,0.0,0.0,1.0,134.0,01-05 05:00,53.8618,8.72498,0.0,0.0,1.0,134.0,01-05 05:00,53.86180,8.72498,0.0,0.0,1.0,134.0,01-05 05:00,53.86180,8.72498,N

In [87]:
ais_test_enriched.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51739 entries, 50000 to 101738
Data columns (total 51 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   time               51739 non-null  datetime64[ns]
 1   vesselId           51739 non-null  object        
 2   scaling_factor     51739 non-null  float64       
 3   sog_lag1           51739 non-null  float64       
 4   cog_lag1           51739 non-null  float64       
 5   rot_lag1           51739 non-null  float64       
 6   heading_lag1       51739 non-null  float64       
 7   etaRaw_lag1        51739 non-null  object        
 8   latitude_lag1      51739 non-null  float64       
 9   longitude_lag1     51739 non-null  float64       
 10  sog_lag2           51739 non-null  float64       
 11  cog_lag2           51739 non-null  float64       
 12  rot_lag2           51739 non-null  float64       
 13  heading_lag2       51739 non-null  float64       
 14  etaRaw

In [ ]:
# Check for null values in each column
null_values = ais_test_enriched.isnull().sum()

# Display the result
print(null_values)